# Test OpenMeteo IFS forecast data integration

In [26]:
from dhis2eo.data.openmeteo.ecmwf_ifs import hourly
import s3fs
from datetime import date

In [27]:
fs = s3fs.S3FileSystem(anon=True)
var = 'temperature_2m'
day = date.fromisoformat('2020-01-01')
ds = hourly.open_ifs_file(
    fs=fs,
    day=day,
    variable=var,
    #chunks={'time': 2},
)
ds

KeyboardInterrupt: 

In [ ]:
import s3fs
#var = 'temperature_2m'
#year = '2026'
url = f"s3://openmeteo/data_spatial/ecmwf_ifs/2026/01/31/0000Z/2026-01-31T0000.om"
fs = s3fs.S3FileSystem(anon=True)
fsfile = fs.open(url)
fsfile

<File-like object S3FileSystem, openmeteo/data_spatial/ecmwf_ifs/2026/01/31/0000Z/2026-01-31T0000.om>

In [ ]:
import xarray as xr
ds = xr.open_dataset(
        fs.open(url),
        engine="om",
        #chunks=None,
    )
ds

RuntimeError: Decoder error Python I/O error KeyboardInterrupt: 

-------------
Direct omfiles use (fails)

In [ ]:
import fsspec
url = f"s3://openmeteo/data_spatial/ecmwf_ifs025/2026/01/31/0000Z/2026-01-31T0000.om"
backend = fsspec.open(
    f"blockcache::{url}",
    mode="rb",
    s3={"anon": True, "default_block_size": 65536},  # s3 settings
    #blockcache={"cache_storage": "cache"},  # blockcache settings
)

In [ ]:
from omfiles import OmFileReader

root = OmFileReader(backend)
print(root)

def iter_children(obj):
    for i in range(obj.num_children):
        ch = obj.get_child_by_index(i)
        yield ch

for ch in iter_children(root):
    print(ch.name, ch.is_group, ch.is_array, ch.is_scalar, ch.num_children)

if root.is_array:
    print('file is an array:', root.shape)

soil_temperature_100_to_255cm False True False 1
total_column_integrated_water_vapour False True False 1
precipitation_type False True False 1
soil_temperature_7_to_28cm False True False 1
soil_temperature_0_to_7cm False True False 1
soil_moisture_0_to_7cm False True False 1
geopotential_height_850hPa False True False 1
sea_ice_thickness False True False 1
relative_humidity_400hPa False True False 1
geopotential_height_100hPa False True False 1
soil_moisture_7_to_28cm False True False 1
wind_u_component_500hPa False True False 1
wind_v_component_500hPa False True False 1
temperature_850hPa False True False 1
soil_moisture_28_to_100cm False True False 1
temperature_100hPa False True False 1
relative_humidity_250hPa False True False 1
wind_v_component_400hPa False True False 1
wind_u_component_400hPa False True False 1
geopotential_height_600hPa False True False 1
soil_moisture_100_to_255cm False True False 1
relative_humidity_150hPa False True False 1
wind_u_component_250hPa False True 

ValueError: Child at index 68 does not exist

In [ ]:
coords = root.get_child_by_name('coordinates')
coords.read_scalar()

'lat lon'

In [ ]:
valid_time = root.get_child_by_name('valid_time')
valid_time.read_scalar()

np.int64(1769817600)

In [ ]:
forecast_time = root.get_child_by_name('forecast_reference_time')
forecast_time.read_scalar()

np.int64(1769817600)

In [ ]:
# get bbox
from pathlib import Path
import geopandas as gpd
DATA_DIR = Path(r'C:\Users\karimba\Documents\Github\dhis2eo\tests\test_data')
geojson_file = DATA_DIR / "sierra-leone-districts.geojson"
org_units = gpd.read_file(geojson_file)
bbox = org_units.total_bounds
xmin,ymin,xmax,ymax = map(float, bbox)

In [ ]:
temp = root.get_child_by_name('temperature_2m')
print(temp.shape)
temp[0:10, 0:10]

ValueError: Child with name 'temperature_2m' does not exist

In [ ]:
import datetime as dt

import fsspec
import numpy as np
from omfiles import OmFileReader

# Example: URI for a spatial data file in the `data_spatial` S3 bucket
# See data organization details: https://github.com/open-meteo/open-data?tab=readme-ov-file#data-organization
MODEL_DOMAIN = "dwd_icon"
# Note: Spatial data is only retained for 7 days. The script uses one file within this period.
date_time = dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=2)
S3_URI = (
    f"s3://openmeteo/data_spatial/{MODEL_DOMAIN}/{date_time.year}/"
    f"{date_time.month:02}/{date_time.day:02}/0000Z/"
    f"{date_time.strftime('%Y-%m-%d')}T0000.om"
)
print(f"Using om file: {S3_URI}")

# Create and open filesystem, wrapping it in a blockcache
backend = fsspec.open(
    f"blockcache::{S3_URI}",
    mode="rb",
    s3={"anon": True, "default_block_size": 65536},  # s3 settings
    blockcache={"cache_storage": "cache"},  # blockcache settings
)
# Create reader from the fsspec file object using a context manager.
# This will automatically close the file when the block is exited.
with OmFileReader(backend) as root:
    # We are at the root of the data hierarchy!
    # What type of node is this?
    print(f"root.is_array: {root.is_array}")  # False
    print(f"root.is_scalar: {root.is_scalar}")  # False
    print(f"root.is_group: {root.is_group}")  # True

    temperature_reader = root.get_child_by_name("temperature_2m")
    print(f"temperature_reader.is_array: {temperature_reader.is_array}")  # True
    print(f"temperature_reader.is_scalar: {temperature_reader.is_scalar}")  # False
    print(f"temperature_reader.is_group: {temperature_reader.is_group}")  # False

    # What shape does the stored array have?
    print(f"temperature_reader.shape: {temperature_reader.shape}")  # (1441, 2879)

    # Read all data from the array
    temperature_data = temperature_reader.read_array((...))
    print(f"temperature_data.shape: {temperature_data.shape}")  # (1441, 2879)

    # It's also possible to read any subset of the array
    temperature_data_subset1 = temperature_reader.read_array((slice(0, 10), slice(0, 10)))
    print(temperature_data_subset1)
    print(f"temperature_data_subset1.shape: {temperature_data_subset1.shape}")  # (10, 10)

    # Numpy basic indexing is supported for direct access if the reader is an array.
    temperature_data_subset2 = temperature_reader[0:10, 0:10]
    print(temperature_data_subset2)
    print(f"temperature_data_subset2.shape: {temperature_data_subset2.shape}")  # (10, 10)

    # Compare the two temperature subsets and verify that they are the same
    are_equal = np.array_equal(temperature_data_subset1, temperature_data_subset2, equal_nan=True)
    print(f"Are the two temperature subsets equal? {are_equal}")

Using om file: s3://openmeteo/data_spatial/dwd_icon/2026/01/30/0000Z/2026-01-30T0000.om
root.is_array: False
root.is_scalar: False
root.is_group: True
temperature_reader.is_array: True
temperature_reader.is_scalar: False
temperature_reader.is_group: False
temperature_reader.shape: (1441, 2879)


RuntimeError: Decoder error Python I/O error PermissionError: [WinError 5] Access is denied: 'c:\\Users\\karimba\\Documents\\Github\\dhis2eo\\dhis2eo\\data\\openmeteo\\ecmwf_ifs\\cache\\cache-vgzfn57x' -> 'cache\\cache'

-----
NEW FOR SEAS5

In [ ]:
from tempfile import gettempdir
import fsspec
import os
url = f"s3://openmeteo/data/ecmwf_seas5_ensemble_mean/temperature_2m/chunk_113.om"
#cache_dir = os.path.abspath('./my_local_cache') #gettempdir()
#os.makedirs(cache_dir, exist_ok=True)
#print(cache_dir)
source = fsspec.open(
    f"blockcache::{url}",
    mode="rb",
    s3={"anon": True, "default_block_size": 65536},  # s3 settings
    blockcache={"cache_storage": None},  # blockcache settings
    check_files=False,       # <--- Crucial: Reduces file handle operations
    expiry_time=False        # Prevents the cache from trying to delete/refresh 
)

# VERY ANNOYING, THE BLOCKCACHE ENDS UP HITTING WINDOWS PERMISSION ERROR LATER WHEN TRYING TO SUBSET THE ARRAY
# ... 

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# 1. Define S3 and Local Paths
url = f"s3://openmeteo/data/ecmwf_seas5_ensemble_mean/temperature_2m/chunk_113.om"
source = os.path.abspath("testdownload.om")

# 2. Direct Download (No caching protocols)
if not os.path.exists(source):
    print(f"Downloading from S3 to {source}...")
    # Use anon=True for Open-Meteo public buckets
    fs = fsspec.filesystem("s3", anon=True)
    fs.get(url, source)
    print("Download finished.")

In [ ]:
from omfiles import OmFileReader

root = OmFileReader(source)
print(root)

print('file is a single snapshot of latest seasonal forecast')
print('dimensions should be (1, flat latlong representing (721, 1440), 720 time chunks)')
print('starttime and other metadata available at https://openmeteo.s3.amazonaws.com/data/ecmwf_seas5_ensemble_mean/static/meta.json')

if root.is_array:
    print('file is an array:', root.shape)

file is a single snapshot of latest seasonal forecast
dimensions should be (1, flat latlong representing (721, 1440), 720 time chunks)
starttime and other metadata available at https://openmeteo.s3.amazonaws.com/data/ecmwf_seas5_ensemble_mean/static/meta.json
file is an array: (1, 421120, 720)


In [ ]:
step_num = 1
first_step = root[0, :, 1]
first_step

array([nan, nan, nan, ..., nan, nan, nan], shape=(421120,), dtype=float32)

----
Finally found an easy xarray way thats efficient

In [ ]:
url = f"s3://openmeteo/data/ecmwf_seas5_ensemble_mean/temperature_2m/chunk_113.om"
#url = f"s3://openmeteo/data_spatial/ecmwf_ifs025/2026/01/31/0000Z/2026-01-31T0000.om"  # THIS ONE IS SLOW
import fsspec
import xarray as xr
cached_path = f"blockcache::{url}"
with fsspec.open(cached_path, mode="rb", s3={"anon": True}) as f:
    ds = xr.open_dataset(f, engine="om")
ds

I guess it works at least, but not so efficient when loading the data

In [ ]:
ds.isel(dim0=0, dim1=slice(0, 100), dim2=0).as_numpy()

<xarray.Dataset> Size: 400B
Dimensions:  (dim1: 100)
Dimensions without coordinates: dim1
Data variables:
             (dim1) float32 400B nan nan nan nan nan nan ... nan nan nan nan nan

In [43]:
first_step = ds.isel(dim2=0) #.to_numpy()
first_step.isel(dim0=0, dim1=slice(0, 100))

<xarray.Dataset> Size: 400B
Dimensions:  (dim1: 100)
Dimensions without coordinates: dim1
Data variables:
             (dim1) float32 400B ...

-----
Interpolate from o320 to regular grid

In [ ]:
from earthkit.regrid import interpolate, db
from earthkit.regrid.interpolate import NumpyInterpolator

In [ ]:
# manually add matrix bc earthkit generates the wrong path
matrix_path = r"C:\Users\karimba\Downloads\caa9775aaa993a68773ff02f19f9c856179383c9e6903f3f7371ed3572ac3692.npz"
#db.add_matrix_source(matrix_path)

In [ ]:
in_grid = {"grid": "O320"}
out_grid = {"grid": [0.25, 0.25]}
out_grid_size = (721, 1440) # should be implied from the grid res
#grid = NumpyInterpolator()(first_step, in_grid=in_grid, out_grid=out_grid, method='linear', matrix_source=matrix_path)
#grid = interpolate(first_step, in_grid=in_grid, out_grid=out_grid, method='linear')
def custom_interpolate(data_flat, matrix_path, out_size):
    from scipy.sparse import load_npz
    # Load the sparse matrix
    # The .npz contains the weights to map 421,120 -> Regular Grid (e.g., 720x1440)
    matrix = load_npz(matrix_path)
    print(matrix)
    # Perform sparse matrix multiplication
    # (Matrix @ Vector)
    interpolated = matrix.dot(data_flat)
    # Reshape to the target grid dimensions (e.g., for 0.25 deg)
    # You'll need to know your target shape (lat_size, lon_size)
    return interpolated.reshape(out_size) 
grid = custom_interpolate(first_step, matrix_path, out_grid_size)
grid

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3108680 stored elements and shape (1038240, 421120)>
  Coords	Values
  (0, 19)	0.5
  (0, 0)	0.5
  (1, 19)	0.5
  (1, 0)	0.5
  (2, 19)	0.5
  (2, 0)	0.5
  (3, 19)	0.5
  (3, 0)	0.5
  (4, 19)	0.5
  (4, 0)	0.5
  (5, 19)	0.5
  (5, 0)	0.5
  (6, 19)	0.5
  (6, 0)	0.5
  (7, 19)	0.5
  (7, 0)	0.5
  (8, 19)	0.5
  (8, 0)	0.5
  (9, 19)	0.5
  (9, 0)	0.5
  (10, 19)	0.5
  (10, 0)	0.5
  (11, 19)	0.5
  (11, 0)	0.5
  (12, 19)	0.5
  :	:
  (1038227, 421119)	0.5
  (1038228, 421100)	0.5
  (1038228, 421119)	0.5
  (1038229, 421100)	0.5
  (1038229, 421119)	0.5
  (1038230, 421100)	0.5
  (1038230, 421119)	0.5
  (1038231, 421100)	0.5
  (1038231, 421119)	0.5
  (1038232, 421100)	0.5
  (1038232, 421119)	0.5
  (1038233, 421100)	0.5
  (1038233, 421119)	0.5
  (1038234, 421100)	0.5
  (1038234, 421119)	0.5
  (1038235, 421100)	0.5
  (1038235, 421119)	0.5
  (1038236, 421100)	0.5
  (1038236, 421119)	0.5
  (1038237, 421100)	0.5
  (1038237, 421119)	0.5
  (1038238, 4211

TypeError: cannot directly convert an xarray.Dataset into a numpy array. Instead, create an xarray.DataArray first, either with indexing on the Dataset or by invoking the `to_dataarray()` method.

In [ ]:
import numpy as np
import xarray as xr
lats = np.arange(90, -90.1, -0.25)
lons = np.arange(0, 360, 0.25) 
da = xr.DataArray(
    grid,
    coords=[lats, lons],
    dims=["lat", "lon"],
    name="temperature_2m",
    attrs={"model": "IFS SEAS5"}
)
da

<xarray.DataArray 'temperature_2m' (lat: 721, lon: 1440)> Size: 8MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(721, 1440))
Coordinates:
  * lat      (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Attributes:
    units:    K
    model:    IFS SEAS5

In [ ]:
np.unique(grid)

array([nan])